In [ ]:
import pandas as pd
import numpy as np

In [ ]:
raw_data = pd.read_csv("cleansed_data.csv")
forecast_data = pd.read_csv("forecast_data.csv")

In [ ]:
raw_data.head()


In [ ]:
forecast_data['Key'] = forecast_data['dc_name'] + '|'+forecast_data['size_code'].astype(str)
forecast_data.head()

In [ ]:
train_data = raw_data[['date','Key','total_tires']]
train_data['date'] = pd.to_datetime(train_data['date'])
forecast_data['date'] = pd.to_datetime(forecast_data['date'])

In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
train_data

In [ ]:
def linear_reg(sliced_data,future_data):


    key = sliced_data['Key'].unique()[0]
    #future_data = forecast_data.copy()

    test_df = future_data[future_data['Key'] == key]
    
    x_train = sliced_data['date'].values.reshape(-1, 1)
    x_train = x_train.astype(float)
    y_train = sliced_data['total_tires'].values

    x_test = test_df['date'].values.reshape(-1, 1)
    x_test = x_test.astype(float)

    
    regressor = LinearRegression() 

    regressor.fit(x_train,y_train)
    

    y_test = regressor.predict(x_test)
    
    test_df['Forecast'] = y_test.round()
    test_df['Model'] = 'Linear_Regression'
    test_df['Type'] = 'Univariate'
    test_df = test_df.drop(columns='Key')
    return test_df


In [ ]:
py_af_output = train_data.groupby('Key').apply(linear_reg,forecast_data).reset_index()

In [ ]:
py_af_output

In [ ]:
linear_result = py_af_output[['date','dc_name','size_code','Forecast']].rename(columns={'Forecast':'total_tires'})
linear_result

In [ ]:
pip install AutoTS

In [ ]:
from autots import AutoTS

In [ ]:
model = AutoTS(
    forecast_length=15,
    frequency='D',
    prediction_interval=0.95,
    ensemble=None,
    models_mode='deep',
    model_list = 'univariate',# or ['ARIMA','ETS']
    max_generations=10,
    num_validations=3,
    no_negatives=True,
    n_jobs='auto')

In [ ]:
from autots.models.model_list import model_lists
print(model_lists.keys())

In [ ]:
print(model_lists['univariate'])

In [ ]:
def auto_ml(sliced_data,future_data):
    model = AutoTS(
    forecast_length=7,
    frequency='D',
    prediction_interval=0.95,
    ensemble=None,
    models_mode='deep',
    model_list = 'univariate',# or ['ARIMA','ETS']
    max_generations=5,
    num_validations=3,
    no_negatives=True,
    n_jobs='auto')

    key = sliced_data['Key'].unique()[0]

    test_df = future_data[future_data['Key'] == key]
    
    sliced_data.set_index('date', inplace=True)
    model.fit(sliced_data['total_tires'])
    prediction = model.predict(forecast_length=7)
    
    test_df['Forecast'] = prediction.forecast['total_tires'].values.tolist()
    test_df['Forecast'] = test_df['Forecast'].round()
    test_df['Model'] = 'Auto_ML_Best_Model'
    test_df['Type'] = 'Univariate'
    test_df = test_df.drop(columns='Key')
    return test_df

In [ ]:
train_data = train_data[train_data['dc_name'] == 'BAKERSFIELD']
auto_output = train_data.groupby('Key').apply(auto_ml,forecast_data).reset_index()

In [ ]:
auto_output = auto_output[['date','dc_name','size_code','Forecast']].rename(columns={'Forecast':'total_tires'})
auto_output

In [ ]:
#import requests
#url = "https://scoring-app-uuzeqpiufa-ue.a.run.app/forecast/validate"
#payload = {
  
#"team_key": "9MaRyrU5",
#"data": df_submission.to_dict(orient="records")}
#response = requests.post(url, json=payload) 
#print(response.status_code, response.content)